In [ ]:
from torchinfo import summary
import model.TinyVGG as tinyVGG
model = tinyVGG.TinyVGG(input_shape=3,
                                hidden_units=10, 
                                output_shape=3)
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
TinyVGG (TinyVGG)                        [32, 3, 224, 224]    [32, 3]              --                   True
├─Sequential (conv_block_1)              [32, 3, 224, 224]    [32, 10, 110, 110]   --                   True
│    └─Conv2d (0)                        [32, 3, 224, 224]    [32, 10, 222, 222]   280                  True
│    └─ReLU (1)                          [32, 10, 222, 222]   [32, 10, 222, 222]   --                   --
│    └─Conv2d (2)                        [32, 10, 222, 222]   [32, 10, 220, 220]   910                  True
│    └─ReLU (3)                          [32, 10, 220, 220]   [32, 10, 220, 220]   --                   --
│    └─MaxPool2d (4)                     [32, 10, 220, 220]   [32, 10, 110, 110]   --                   --
├─Sequential (conv_block_2)              [32, 10, 110, 110]   [32, 10, 53, 53]     --                   True
│    └─Conv2d (0)   

In [15]:
import torch
import torch.nn as nn
import torchvision
from torchinfo import summary


# Set up the device to use GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note: This is how a pretrained model would be created in torchvision > 0.13, it will be deprecated in future versions.
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # OLD 

# Download the pretrained weights for EfficientNet_B0
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # NEW in torchvision 0.13, "DEFAULT" means "best weights available"

# Setup the model with the pretrained weights and send it to the target device
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

# View the output of the model
model
# Freeze all base layers by setting requires_grad attribute to False
for param in model.features.parameters():
    param.requires_grad = False
    
# Since we're creating a new layer with random weights (torch.nn.Linear), 
# let's set the seeds
random_seed = 42
torch.manual_seed(random_seed)

# Update the classifier head to suit our problem
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=3, # 3 classes for our problem
              bias=True).to(device))


# Get a summary of the model (uncomment for full output)
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 